In [ ]:
import pandas as pd
import pdb
import sys
import os
import re
import glob
import multiprocessing as mp
import numpy as np 
import uuid
import numpy.random as npr
from datetime import datetime, timedelta
import math
import itertools
import copy
import time
import scipy
import scipy.stats as sps
from progressbar import ProgressBar

import pyarrow as pa
import pyarrow.parquet as pq
import s3fs
s3 = s3fs.S3FileSystem()
import geopandas as gpd

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

# Epidemic modeling 

In [ ]:
def initialize_states(pop_geoid, params,model='SEIR'):
    if 'S' not in params.keys():
        sys.exit("Initial conditions must be in params to use this method")
    states = pd.DataFrame(index=pop_geoid.index)
    states['N'] = pop_geoid['pop'].copy() #total population
    probs = states['N']/np.sum(states['N'])

    N_minus_S = np.sum(states['N']) - params['S']
    N_minus_S  = npr.multinomial(int(N_minus_S), pvals = probs) 

    states['S'] = states['N'] - N_minus_S 
    states['S'] = states[['N','S']].min(axis=1)
    states['S/N'] = states['S']/states['N']

    states['E'] = npr.multinomial(int(params['E']), pvals = probs) 
    states['E'] = states[['N','E']].min(axis=1)

    states['I'] = npr.multinomial(int(params['I']), pvals = probs) 
    states['I'] = states[['N','I']].min(axis=1)

    if model == 'SEIIR':
        states['I_s'] = (states['I']*params['rho']).round()
        states['I_a'] = (states['I']*(1-params['rho'])).round()
        states['I_s/N'] = states['I_s']/states['N']
        states['I_a/N'] = states['I_a']/states['N']
        states['I'] = None
    elif model == 'SEIR':
        states['I/N'] = states['I']/states['N']
    else:
        sys.exit('model not programmed')
    return(states)

In [ ]:
def compute_rate_SEIR(df, beta):
    edge_rate = beta*df['S/N']*df['I/N']*df['expected_contacts_sq_ft']
    effective_rate = pd.Series(edge_rate,name='rate')
    effective_rate['origin_geoid'] = df['origin_geoid']
    effective_rate = effective_rate.groupby('origin_geoid').sum()
    return(effective_rate)

In [ ]:
def state_transitions(states, effective_rates, params,model='SEIR', log_p=False):   
    tr = pd.DataFrame(index=states.index)
    #First fill a dataframe with rates
#     print('In S_E part 1:',effective_rates.isnull().sum())
#     print(tr.shape,len(effective_rates))
    tr['S_to_E'] = effective_rates
#     print('In S_E:',tr['S_to_E'].isnull().sum())
    tr['E_to_I'] = params['kappa']*states['E']
    if model == 'SEIR':
        tr['I_to_R'] = params['gamma']*states['I']
        if log_p: rates = tr.copy()
#         print(tr[['S_to_E', 'E_to_I', 'I_to_R']].isnull().sum())
#         print(tr[['S_to_E', 'E_to_I', 'I_to_R']].shape)
        tr.loc[:,['S_to_E', 'E_to_I', 'I_to_R']] = npr.poisson(
            tr[['S_to_E', 'E_to_I', 'I_to_R']])
        if log_p:
            p=np.sum(sps.poisson.logpmf(mu=rates, k=tr))
            return(tr, p)
        else:
            return(tr)
    elif model == 'SEIIR':
        tr['Is_to_R'] = params['gamma']*states['I_s']
        tr['Ia_to_R'] = params['gamma']*states['I_a']
        #Sample Poissons with the array of rates all at once.
        tr.loc[:,['S_to_E', 'E_to_I', 'Is_to_R', 'Ia_to_R']] = npr.poisson(
            tr[['S_to_E', 'E_to_I', 'Is_to_R', 'Ia_to_R']])

        tr['S_to_E'] = tr[['S_to_E', 'S']].min(axis=1)
        tr['E_to_I'] = tr[['E_to_I', 'E']].min(axis=1)
        tr['Is_to_R'] = tr[['Is_to_R', 'I_s']].min(axis=1)
        tr['Ia_to_R'] = tr[['Ia_to_R', 'I_a']].min(axis=1)
        return(tr)
    else:
        sys.exit('model not programmed')

In [ ]:
def apply_transitions(states, trans, params, model='SEIR'):
    trans = pd.concat([trans, states[['S', 'E', 'I']]], axis=1)
    trans['S_to_E'] = trans[['S_to_E', 'S']].min(axis=1)
    trans['E_to_I'] = trans[['E_to_I', 'E']].min(axis=1)
    trans['I_to_R'] = trans[['I_to_R', 'I']].min(axis=1)

    states['S'] += -trans['S_to_E']
    states['E'] += trans['S_to_E'] - trans['E_to_I'] 
    if model == 'SEIR':
        states['I'] += trans['E_to_I'] - trans['I_to_R']
        states['I/N'] = states['I']/states['N']
    elif model == 'SEIIR':
        states['I_s'] += (params['rho']*trans['E_to_I']).round() - trans['Is_to_R']
        states['I_a'] += ((1. -params['rho'])*trans['E_to_I']).round() - trans['Ia_to_R'] 
        states['I_s/N'] = states['I_s']/states['N']
        states['I_a/N'] = states['I_a']/states['N']
    else:
        sys.exit('model not programmed')
    states['S/N'] = states['S']/states['N']
    return(None)

In [ ]:
def update_states(states, params, net, model='SEIR', log_p=False):
    """
    Takes the states of the model in a given day and simulates the 
    compartmental transitions. The only non-static parameter, is the 
    contact network. 
    params:
        states: dict
            current compartment counts for every subpopulation.
        alpha: float
            discount factor for contact with an asymptomatic infective.
        beta: float
            rate of infection the probability of a susceptible becoming 
            infected from exposure to one infective/sq_foot. 
        gamma: float
            rate at which infectives recover.
        kappa: float
            rate at which an exposed becomes infective.
        rho: float
            probability of exposed becoming symptomatic.
        net: pandas DataFrame
            network estimating the number of contacts with other CBGs
            in a given day.
        net: dict
            contact network with double index (origin_geoid, dest_geoid) 
        model: str
            the type of compartmental model
        out_trans: bool
            if trans, we return the transitions
            for likelihood computation
    """
    #compute effective infection rates
    net = net.join(states['S/N'], on='origin_geoid')
    if model == 'SEIIR':
        net = net.join(
            states[['I_a/N', 'I_s/N']],
            on='destination_geoid')
        effective_rates = compute_rate(
            net,
            states=states,
            alpha=params['alpha'],
            beta=params['beta'])
    elif model == 'SEIR':
        net = net.join(states['I/N'], on='destination_geoid')
        effective_rates = compute_rate_SEIR(
            net,
            beta=params['beta'])
    #sample state transitions
#     print('123 count nulls:',effective_rates.isnull().sum())
    if log_p:
        trans, p = state_transitions(states, effective_rates, params, model, log_p)
        apply_transitions(states, trans, params, model)
        return(states, trans['E_to_I'], p)
    else:
        trans = state_transitions(states, effective_rates, params, model)
        apply_transitions(states, trans, params, model)
        return((states, trans['E_to_I']))

In [ ]:
def get_net(path, date, index_dict, net_type='fsq'):
    
    pivot_table = pd.read_csv(f'{path}{date}.csv').iloc[:,1:] # ct_to_ct
#     print('Shape heres:',pivot_table.shape)
    net = pd.melt(pivot_table, id_vars=['home_CT'], var_name='column_name', value_name='value')
    net = net.rename(columns={'home_CT':'origin_geoid','column_name':'destination_geoid','value':'expected_contacts'})
    
    # filter those in pop_df.index
    net = net[(net.origin_geoid.isin(pop_df.index)) & (net.destination_geoid.isin(pop_df.index))]
    
    net['origin_geoid'] = net.origin_geoid.astype(str)
    net['destination_geoid'] = net.destination_geoid.astype(str)
    return net

In [ ]:
def run_model(params,
              path,
              pop_geoid,
              index_dict,
              net_type,
              date_interval,
              model='SEIR',
              num_sims=36,
              initial_cond=None):
    
#     caseload_df = pd.DataFrame({'date':date_interval})
    caseload_df = pd.DataFrame(columns=['iterate', 'date', 'census_tract', 'new_cases'])

    pbar = ProgressBar()
    for i in pbar(range(num_sims)):
        if initial_cond is not None:
            states = initialize_states_list(
                initial_cond,
                i,
                pop_geoid)
        else:
            states = initialize_states(
                pop_geoid,
                params=params)
        
        caseload = []
        patterns_date = ''
        for date in date_interval:
            print(f"Simulating for date {date}")

            net=get_net(path, date, index_dict, net_type)
            net = net.merge(pop_geoid.reset_index(), left_on='destination_geoid',right_on='index',how='left')[list(net.columns) + ['area']]
            net['expected_contacts_sq_ft'] = net.expected_contacts / net.area

            states, new_cases = update_states(
                states=states,
                params=params,
                net=net)
#             caseload.append(new_cases) # new_cases 
            for ct, cases in zip(new_cases.index, new_cases.values):
                caseload_df = caseload_df.append({'iterate': i, 'date': date, 'census_tract': ct, 'new_cases': cases}, ignore_index=True)

    # Set the index of caseload_df
#     caseload_df.set_index(['date', 'census_tract'], inplace=True)

#         caseload_df[i]=caseload
#     caseload_df.set_index('date',inplace=True)
#     caseload_df = caseload_df.groupby(['date','census_tract']).mean()
    caseload_df['new_cases'] = pd.to_numeric(caseload_df['new_cases'], errors='coerce')
#     caseload_df = caseload_df.groupby(['date','census_tract']).mean()
#     new_df = df.pivot(index='Index1', columns='Value', values='Value').fillna(0)

    caseload_df = caseload_df.pivot(index=['date','census_tract'], columns='iterate', values='new_cases').fillna(0)

    return (caseload_df)

In [ ]:
# get populations
def get_pop():
    cbg_pops = pd.read_csv('./tmp/populations_cbg.csv')

    cbgs = gpd.read_file("./tmp/Census_Block_Groups_2010.geojson", crs=4326)
    cbgs = cbgs.loc[cbgs.GEOID10.apply(lambda x: x[:5] == "42101"), ["GEOID10","geometry", "Shape__Area"]]

    cbg_pops['GEOID'] = cbg_pops.GEOID.astype(str)
    cbg_pops = cbg_pops.loc[cbg_pops.GEOID.apply(lambda x: x[:5] == "42101")]
    cbg_pops['CT'] = cbg_pops.GEOID.apply(lambda x: x[:-1])
    cbg_pops = cbg_pops.merge(cbgs, left_on='GEOID', right_on='GEOID10',how='left')

    ct_pops = cbg_pops.groupby('CT')[['B01003_001_Population', 'Shape__Area']].sum().reset_index()

    pop_dict = ct_pops.set_index('CT').to_dict()

    pop_df = pd.DataFrame.from_dict(pop_dict)
    pop_df = pop_df.rename({'B01003_001_Population':'pop', 'Shape__Area':'area'},axis=1).sort_index()
    return pop_df

# Run epidemic simulation

In [ ]:
params = {
        'beta' : 2000,
        'kappa' : 0.22,
        'gamma' : 0.14,
        'tau' : 0.08
}

start = datetime.strptime("04-01-2020", "%m-%d-%Y")
end = datetime.strptime("04-30-2020", "%m-%d-%Y")
date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days)]
date_interval = [date.strftime("%Y-%m-%d") for date in date_generated]

path = "s3://phl-poi-networks/fsq/"
net_type = "fsq"

pop_df = get_pop()

bip = pd.read_csv((f'{path}{date_interval[0]}.csv'), index_col=0).reset_index()
index_dict = bip['home_CT'].to_dict()

# restrict pop_geoid to just CTs in bip
pop_df = pop_df[pop_df.index.isin(bip.home_CT.astype(str))]

philly_pop = pop_df['pop'].sum()
params['E'] = 100
params['S'] = philly_pop - params['E']
params['I'] = 0

test_fsq = run_model(params, path, pop_df, index_dict, net_type, date_interval, num_sims=5)

In [ ]:
date_sum_fsq = test_fsq.groupby('date').sum()
date_sum_fsq

In [ ]:
params = {
        'beta' : 2000,
        'kappa' : 0.22,
        'gamma' : 0.14,
        'tau' : 0.08
}

start = datetime.strptime("04-01-2020", "%m-%d-%Y")
end = datetime.strptime("04-30-2020", "%m-%d-%Y")
date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days)]
date_interval = [date.strftime("%Y-%m-%d") for date in date_generated]

path = "s3://phl-poi-networks/stanford/"
net_type = "sg"

pop_df = get_pop()

bip = pd.read_csv((f'{path}{date_interval[0]}.csv'), index_col=0).reset_index()
index_dict = bip['home_CT'].to_dict()

# restrict pop_geoid to just CTs in bip
pop_df = pop_df[pop_df.index.isin(bip.home_CT.astype(str))]

philly_pop = pop_df['pop'].sum()
params['E'] = 100
params['S'] = philly_pop - params['E']
params['I'] = 0

test_sg = run_model(params, path, pop_df, index_dict, net_type, date_interval, num_sims=5)

In [ ]:
date_sum_sg = test_sg.groupby('date').sum()
date_sum_sg

In [ ]:
quantile_fsq = np.quantile(date_sum_fsq, [0.1, 0.5, 0.9], axis=1).transpose()
quantile_fsq = pd.DataFrame(quantile_fsq, columns = ['low_fsq', 'median_fsq', 'high_fsq'], index=date_sum_fsq.index)
quantile_fsq

In [ ]:
quantile_sg = np.quantile(date_sum_sg, [0.1, 0.5, 0.9], axis=1).transpose()
quantile_sg = pd.DataFrame(quantile_sg, columns = ['low_sg', 'median_sg', 'high_sg'], index=date_sum_sg.index)
quantile_sg

In [ ]:
quantile = quantile_fsq.join(quantile_sg)

In [ ]:
x_axis = quantile.index
fig, ax = plt.subplots()

ax.plot(x_axis,
    quantile['median_fsq'],
    marker='',
    color='skyblue',
    linewidth=2,
    label = 'fsq')
plt.legend(loc='upper right')

#plot confidence
plt.fill_between(
    x_axis,
    quantile['low_fsq'],
    quantile['high_fsq'],
    color='b',
    alpha=.1)

ax.plot(x_axis,
    quantile['median_sg'],
    marker='',
    color='red',
    linewidth=2,
    label = 'sg')
plt.legend(loc='upper right')

#plot confidence
plt.fill_between(
    x_axis,
    quantile['low_sg'],
    quantile['high_sg'],
    color='r',
    alpha=.1)
plt.title('Simulated epidemic (PHL)', fontsize=18)
plt.grid()

fig.set_size_inches(10, 5)
# fig.autofmt_xdate()
# xfmt = mdates.DateFormatter('%m-%d')
# ax.xaxis.set_major_formatter(xfmt)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Assuming you have already defined the x_axis and quantile DataFrame

fig, ax = plt.subplots()

ax.plot(x_axis,
        quantile['median_fsq'],
        marker='',
        color='skyblue',
        linewidth=2,
        label='fsq')
plt.legend(loc='upper right')

# Plot confidence
plt.fill_between(
    x_axis,
    quantile['low_fsq'],
    quantile['high_fsq'],
    color='b',
    alpha=.1)

ax.plot(x_axis,
        quantile['median_sg'],
        marker='',
        color='red',
        linewidth=2,
        label='sg')
plt.legend(loc='upper right')

# Plot confidence
plt.fill_between(
    x_axis,
    quantile['low_sg'],
    quantile['high_sg'],
    color='r',
    alpha=.1)

plt.title('Simulated epidemic (PHL)', fontsize=18)
plt.grid()

fig.set_size_inches(10, 5)

# Set the x-axis limits from April 1st to April 29th
start_date = pd.to_datetime('2022-04-01')
end_date = pd.to_datetime('2022-04-29')
plt.xlim(start_date, end_date)

# Customize x-axis tick labels to show dates at weekly intervals
ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

plt.show()


In [ ]:
test_fsq.mean(axis=1).plot()

In [ ]:
test_sg.mean(axis=1).plot()

In [ ]:
population = pd.read_csv('s3://upenn-seas-wattscovid19lab/paco/acs_vars/safegraph_open_census_data/data/data/cbg_b01.csv', dtype={'census_block_group':str})
# Shorten census_block_group to census_tract, convert population to int and group by census_tract
population = population.assign(
    census_tract = population['census_block_group'].str[:-1],
    population = population['B01003e1'].astype(int)).groupby('census_tract')['population'].sum()

population = population[population.index.str.startswith('42101')&(population>5)]

In [ ]:
population

In [ ]:
new_cases = test_fsq.iloc[0,0]

In [ ]:
cbg_pops = pd.read_csv('./tmp/populations_cbg.csv')

cbgs = gpd.read_file("s3://upenn-seas-wattscovid19lab/paco/geometry/Census_Tracts_2010.geojson", crs=4326)
cbgs = cbgs.loc[cbgs.GEOID10.apply(lambda x: x[:5] == "42101"), ["GEOID10","geometry"]]

cbg_pops['CT'] = cbg_pops.GEOID.astype(str).apply(lambda x: x[:-1])

ct_pops = cbg_pops.groupby('CT')['B01003_001_Population'].sum().reset_index()

pop_dict = ct_pops.set_index('CT').to_dict()['B01003_001_Population']

pop_df = pd.DataFrame.from_dict(pop_dict, orient='index', columns=['pop']).reset_index().rename({'index':'home_CT'},axis=1)

In [ ]:
cbgs

In [ ]:
cbg_plot = 

In [ ]:
# plot by visits scaled by CT
phila_poly = gpd.GeoDataFrame(geometry= gpd.GeoSeries([cbgs.geometry.unary_union.buffer(0.000008)], crs=4326))
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(24,10))
fig.suptitle(f'Heatmap of Normalized Visits on April 1 (Chang)',fontsize=24)

cmap='YlOrRd'
divnorm=colors.TwoSlopeNorm(vmin=cbg_plot.scaled_visits.min(), vcenter=cbg_plot.scaled_visits.quantile(0.5), vmax=cbg_plot.scaled_visits.max())
phila_poly.plot(facecolor="lightgray", edgecolor="white", ax=ax)
cbg_plot.plot(column='scaled_visits', cmap=cmap, norm=divnorm, ax=ax)
ax.set_xticks([], minor=False)
ax.set_yticks([], minor=False)

cbar = plt.cm.ScalarMappable(norm=divnorm, cmap=cmap)
cbar_ax = fig.add_axes([0.62, 0.05, 0.15, 0.86])
cbar_ax.set_axis_off()
cbar = fig.colorbar(cbar, ax=cbar_ax, extend='both')
cbar.set_label('Scaled Visits', fontsize=24, rotation=270, labelpad=20)

plt.tight_layout()
plt.savefig('chang_apr1.png')